In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

c:\InstallProgram\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1054)
           2	LOAD_FAST(arg=0, lineno=1054)
           4	LOAD_CONST(arg=1, lineno=1054)
           6	BINARY_SUBSCR(arg=None, lineno=1054)
           8	LOAD_FAST(arg=0, lineno=1054)
          10	LOAD_CONST(arg=2, lineno=1054)
          12	BINARY_SUBSCR(arg=None, lineno=1054)
          14	COMPARE_OP(arg=4, lineno=1054)
          16	LOAD_FAST(arg=0, lineno=1054)
          18	LOAD_CONST(arg=1, lineno=1054)
          20	BINARY_SUBSCR(arg=None, lineno=1054)
          22	LOAD_FAST(arg=0, lineno=1054)
          24	LOAD_CONST(arg=3, lineno=1054)
          26	BINARY_SUBSCR(arg=None, lineno=1054)
          28	COMPARE_OP(arg=5, lineno=1054)
          30	BINARY_AND(arg=None, lineno=1054)
          32	RETURN_VALUE(arg=None, lineno=1054)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=105

## Single Speaker

In [3]:
# 修改文件名字
hps = utils.get_hparams_from_file("configs/ru.json")
import text
text.symbols=symbols
symbols=hps['symbols']

a


In [9]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./models/model.pth", net_g, None)

INFO:root:Loaded checkpoint './path/to/model.pth' (iteration None)


In [5]:
stn_tst = get_text("こんにちは，初めましておめでとうございます", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
au=ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False)
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not NoneType

## Multiple Speakers

In [2]:
hps = utils.get_hparams_from_file("configs/ru.json")
symbols=hps['symbols']
import text
text.symbols=symbols
text._symbol_to_id, text._id_to_symbol=text.changeSymbol(symbols)

In [3]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()


_ = utils.load_checkpoint("./models/model.pth", net_g, None)

INFO:root:Loaded checkpoint './models/model.pth' (iteration 419)


In [9]:
stn_tst = get_text("[ZH]天青色等烟雨，而我在等你[ZH]", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([79]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
au=ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False)
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [8]:
with open('../cache/audio/audio.wav', 'wb') as f:
    f.write(au.data)

### Voice Conversion

In [20]:
# 由于config自带的validation_files与本项目路径不一致，这里自行设置修改
# dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
# 由于找不到原始音频文件，这部分没法做
dataset = TextAudioSpeakerLoader('./filelists/cjke_val_filelist.txt.cleaned', hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'voistock/5d0cd0d43e418659c0492aed.wav'

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))